In [0]:
import requests
import os
from pyspark.sql import SparkSession


In [0]:
# Imports necessários
from pyspark.sql import SparkSession
import requests
import os

# Função para enviar mensagem para o Telegram com base nos dados da tabela gold.insights_diarios
def enviar_insights_telegram():
    try:
        spark = SparkSession.builder.getOrCreate()

        # Lê o último insight salvo
        df = spark.sql("SELECT * FROM gold.insights_diarios ORDER BY data_coleta DESC LIMIT 1")
        linha = df.collect()[0]

        # Constrói a mensagem para o Telegram
        mensagem = f"""
📊 *Relatório Diário - {linha['data_coleta']}*

💰 Produto mais caro: {linha['top_1_produto_caro']} – R$ {linha['preco_produto_caro']:.2f}
🔻 Produto mais barato: {linha['top_1_produto_barato']} – R$ {linha['preco_produto_barato']:.2f}

📈 Média dos preços: R$ {linha['media_preco_dia']:.2f}
📉 Variação média dos preços: R$ {linha['variacao_preco_medio']:.2f}
💹 Variação ação Nvidia: R$ {linha['variacao_nvidia']:.2f}

✅ Dados coletados automaticamente pelo pipeline no Databricks.
"""

        # Definir o Token e Chat ID
        telegram_token = "7801556224:AAHWO1NK04OLYxL2rvNwdhk_m7Fd7xTTG8c"
        chat_id = "7723833482"

        # Construir a URL e o Payload
        url = f"https://api.telegram.org/bot{telegram_token}/sendMessage"
        payload = {
            "chat_id": chat_id,
            "text": mensagem,
            "parse_mode": "Markdown"
        }

        # Enviar a mensagem
        response = requests.post(url, data=payload)
        if response.status_code == 200:
            print("✅ Mensagem enviada com sucesso para o Telegram!")
        else:
            print(f"⚠️ Erro ao enviar mensagem: {response.text}")

    except Exception as e:
        print(f"❌ Erro na função de envio para o Telegram: {str(e)}")

# Função principal para orquestrar a execução dos notebooks
def executar_pipeline():
    try:
        print("▶️ Rodando notebook de Configurações...")
        dbutils.notebook.run("/ecommerce/00.config/configurações", timeout_seconds=600)

        print("▶️ Rodando Bronze - Mercado Livre...")
        dbutils.notebook.run("/ecommerce/01.bronze/bronze_mercado_livre", timeout_seconds=600)

        print("▶️ Rodando Bronze - Kabum...")
        dbutils.notebook.run("/ecommerce/01.bronze/bronze_kabum", timeout_seconds=600)

        print("▶️ Rodando Bronze - Cotação Nvidia...")
        dbutils.notebook.run("/ecommerce/01.bronze/bronze_cotação_nvidia", timeout_seconds=600)

        print("▶️ Rodando Silver...")
        dbutils.notebook.run("/ecommerce/02.silver/silver", timeout_seconds=600)

        print("▶️ Rodando Gold...")
        dbutils.notebook.run("/ecommerce/03.gold/gold", timeout_seconds=600)

        print("🚀 Enviando relatório para o Telegram...")
        enviar_insights_telegram()

        print("🎯 Pipeline executada com sucesso!")

    except Exception as e:
        print(f"❌ Erro na execução da pipeline: {str(e)}")

# Executar toda a orquestração
executar_pipeline()
